In [13]:
import sys
import json
sys.path.append('..')
from src.analysis.model_performances import *
import pandas as pd
EVERYTHING_POSTFIX = 'everything'

In [14]:
def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for question_category in QUESTION_CATEGORIES + [ALL_QUESTION_CATEGORIES_KEY]:
            for ramifications in RAMIFICATION_TYPES:
                index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
                data_columns = {}
                for domain in domains:
                    data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
                data.append(data_columns)
    return pd.DataFrame(data, index = index)

# def df_to_latex_table(df):
#         latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
#         return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

# def assemble_table(results_all, answer_type, domain, score_key=None):
#     latex_table_all = ''
#     with open('latex_table_template/top') as f:
#         latex_table_all += f.read() + '\n'
#     latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
#     with open('latex_table_template/bottom') as f:
#         latex_table_all += f.read()
#     
#     caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
#     latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
#     
#     return latex_table_all


In [15]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = F1_SCORE_KEY)
stats_all = collect_stats_all(tf_answer_type(score_key = F1_SCORE_KEY))

100%|██████████| 15360/15360 [00:02<00:00, 7479.44it/s] 


In [23]:
model_name = 'gemini'#'gemma-2b-it'
prompt_type = 'few_shot_1'
subs = WITHOUT_RANDOM_SUB
domains = DOMAIN_NAMES + [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
# domains = [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
save_key = 'all'#'transport_vs_other'
df = to_df(stats_all, model_name, answer_type, prompt_type, subs, domains)

In [24]:
df

,blocksworld,depots,driverlog,goldminer,grippers,logistics,miconic,mystery,npuzzle,satellite,spanner,visitall,zenotravel,TRANSPORTATION,NON_TRANSPORTATION,ALL_DOMAINS
"(1, object_tracking, R, T/F)",0.571429,0.584270,0.474026,0.562500,0.492857,0.500000,0.621951,0.475000,0.541401,0.512563,0.509524,0.606299,0.494253,0.519251,0.545629,0.532967
"(1, object_tracking, No R, T/F)",0.564286,0.595960,0.487013,0.562500,0.492857,0.491667,0.597561,0.458333,0.522293,0.492462,0.500000,0.606299,0.494253,0.515963,0.536023,0.526342
"(1, fluent_tracking, R, T/F)",0.507407,0.487654,0.512195,0.516667,0.557143,0.550000,0.582456,0.533333,0.507407,0.472222,0.540625,0.544444,0.518519,0.536856,0.513514,0.524091
"(1, fluent_tracking, No R, T/F)",0.492593,0.522222,0.524390,0.502778,0.557143,0.538889,0.585965,0.511111,0.511111,0.469444,0.543750,0.522222,0.514815,0.538362,0.505946,0.520729
"(1, state_tracking, R, T/F)",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(19, numerical_reasoning, No R, T/F)",0.470000,0.500000,0.500000,0.495000,0.470000,0.495000,0.500000,0.480000,0.460000,0.500000,0.500000,0.495000,0.455000,0.485714,0.486667,0.486154
"(19, hallucination, R, T/F)",0.450000,0.460000,0.458333,0.400000,0.341667,0.360000,0.414286,0.380000,0.450000,0.364286,0.335714,0.466667,0.425000,0.406250,0.407692,0.406962
"(19, hallucination, No R, T/F)",0.483333,0.450000,0.433333,0.385714,0.358333,0.370000,0.435714,0.340000,0.450000,0.357143,0.335714,0.450000,0.416667,0.402500,0.406410,0.404430
"(19, all_categories, R, T/F)",0.499462,0.498630,0.503808,0.496409,0.452910,0.508974,0.461679,0.500000,0.510053,0.439640,0.462281,0.530435,0.496373,0.487047,0.487496,0.487271


In [25]:
latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)

latex_table_all = r"""
\begin{table*}[h!]
\begin{adjustbox}{width=\textwidth,center}
""" + latex_table + """
\end{adjustbox}
\caption{}
\end{table*}
"""
os.makedirs(os.path.join(STATISTICS_PATH, 'tables_2'), exist_ok=True)
with open(os.path.join(STATISTICS_PATH, 'tables_2', f'{model_name}.{prompt_type}.{answer_type_ext}.{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_80015/3246921699.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [32]:
latex_table_all

'\n\\begin{table*}[h!]\n\\begin{adjustbox}{width=2\\columnwidth,center}\n\\begin{tabular}{lrrr}\n\\toprule\n{} &  TRANSPORTATION &  NON\\_TRANSPORTATION &  ALL\\_DOMAINS \\\\\n\\midrule\n(1, object\\_tracking, R, T/F)          &            0.03 &                0.04 &         0.03 \\\\\n(1, object\\_tracking, No R, T/F)       &            0.02 &                0.02 &         0.02 \\\\\n(1, fluent\\_tracking, R, T/F)          &            0.19 &                0.15 &         0.17 \\\\\n(1, fluent\\_tracking, No R, T/F)       &            0.18 &                0.14 &         0.16 \\\\\n(1, state\\_tracking, R, T/F)           &            0.42 &                0.40 &         0.41 \\\\\n(1, state\\_tracking, No R, T/F)        &            0.40 &                0.40 &         0.40 \\\\\n(1, action\\_executability, R, T/F)     &            0.18 &                0.09 &         0.14 \\\\\n(1, action\\_executability, No R, T/F)  &            0.16 &                0.08 &         0.12 \\\\\n(1, e